## Train Multiclass Classifier: BERT

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModelForMaskedLM, TrainingArguments, Trainer
from datasets import Dataset, load_dataset, load_from_disk, concatenate_datasets
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from collections import Counter
import random
import numpy as np
import torch
import os

/home/jschelb/.pyenv/versions/3.10.8/envs/s2j-content-analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
topics = ["cannabis", "energie", "kinder"]

## Load Dataset

**Map class-names to class-ids:**

In [3]:
id_to_class = {0: "other",1: "cannabis", 2: "energie", 3: "kinder"}
class_to_id = {"other": 0, "cannabis": 1, "energie": 2, "kinder": 3}

In [4]:
MAX_CONTENT_LENGTH = 384
file_path = f"../data/tmp/processed_dataset_multiclass_chunkified_{MAX_CONTENT_LENGTH}"
dataset = load_from_disk(file_path)

dataset = dataset["train"]

In [5]:
dataset

Dataset({
    features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'token_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'chunk_id', 'url_path'],
    num_rows: 15828
})

## Load Model

In [6]:
model_path = "../models/bert_multiclass_model_buff_incr_neg"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path).eval()

In [7]:
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs!")
    model = torch.nn.DataParallel(model)

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Using 2 GPUs!


DataParallel(
  (module): XLMRobertaForSequenceClassification(
    (roberta): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250002, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): XLMRobertaSelfOutput(


## Prepare Dataset

In [8]:
# Tokenize the text
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) # TODO: URL REMOVED

dataset = dataset.map(tokenize_function, batched=True)

## Get Predictions

In [9]:
def predict_batch(batch):
    """ Perform prediction on a batch of samples in a multiclass setting. """
    
    # Ensure input tensors are on the correct device
    input_ids = torch.tensor(batch['input_ids']).to(device)
    attention_mask = torch.tensor(batch['attention_mask']).to(device)
    
    # Perform prediction
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Extract probabilities for all classes and predicted classes
    batch['probas'] = predictions.cpu().tolist()  # Move results back to CPU and convert to list
    batch['preds'] = torch.argmax(predictions, dim=-1).cpu().tolist()

    return batch


In [10]:
dataset = dataset.map(predict_batch, batched=True, batch_size=512)

In [11]:
id = 100
print(dataset[id]["label"])
print(dataset[id]["preds"])
print(dataset[id]["probas"])

1
1
[0.05970004200935364, 0.939585268497467, 0.0005707441014237702, 0.00014389595889952034]


In [12]:
#dataset.save_to_disk(file_path + "_preds")

## Compare Before and After Filtering

In [13]:
from collections import Counter
from tqdm import tqdm

**Count Difference in Chunk Level Predictions:**

In [14]:
preds = dataset["preds"]
labels = dataset["label"]

In [15]:
print(Counter(preds))
print(Counter(labels))

Counter({0: 10177, 2: 2363, 1: 1736, 3: 1552})
Counter({0: 8493, 2: 2652, 1: 2504, 3: 2179})


**Compare URL Level Predictions (loss of Acc after self-training):**

In [16]:
# Group dataset examples by URL, with a fallback to domain
grouped_dataset = {}
for example in tqdm(dataset):
    url = example.get("view_url") or example.get("domain")
    example_filtered = {k: example[k] for k in ["text", "domain", "probas", "preds", "label", "category", "annotation_type", "lang"]}
    grouped_dataset.setdefault(url, []).append(example_filtered)

100%|██████████| 15828/15828 [00:09<00:00, 1701.85it/s]


In [17]:
# Extract labels
labels_for_url = []
for url, chunks in grouped_dataset.items():
    preds = [chunk["label"] for chunk in chunks]
    labels_for_url.append(max(preds))

In [18]:
def majority_voting(answers):
    """Apply majority voting to a list of arbitrary classification answers."""
    count = Counter(answers)
    most_common = count.most_common()  # Get all common answers sorted by frequency

    if not most_common:
        return 0 # Handle empty input scenario

    # Check for ties at the highest count
    max_votes = most_common[0][1]
    tied_classes = [cls for cls, votes in most_common if votes == max_votes]

    if len(tied_classes) > 1:
        return max(tied_classes)  # Return the maximum class label in case of a tie
    return tied_classes[0]  # Return the class with the most votes

majority_voting([1, 1, 2, 2, 2, 3])

2

In [19]:
prediction_for_url_max = []

# It is kinda nuts that max() works best but the intuition is that most chunks are other and as soon there is anything higher than other (class label 0) it is always related to that policy
for url, chunks in grouped_dataset.items():
    preds = [chunk["preds"] for chunk in chunks]
    #preds = preds if len(preds) > 0 else [0]
    pred = majority_voting([pred for pred in preds if pred > 0]) if max(preds) > 0 else 0
    prediction_for_url_max.append(pred)

In [20]:
# Assuming labels and preds are lists or arrays containing the true labels and predicted labels respectively
accuracy = accuracy_score(labels_for_url, prediction_for_url_max)
precision_per_class = precision_score(labels_for_url, prediction_for_url_max, average=None)
recall_per_class = recall_score(labels_for_url, prediction_for_url_max, average=None)
f1_per_class = f1_score(labels_for_url, prediction_for_url_max, average=None)

print("Overall Accuracy: {:.2f}%".format(accuracy * 100))
print("Precision per class: {}".format(np.round(precision_per_class, 2)))
print("Recall per class: {}".format(np.round(recall_per_class, 2)))
print("F1 Score per class: {}".format(np.round(f1_per_class, 2)))

Overall Accuracy: 93.24%
Precision per class: [0.98 0.95 0.78 0.94]
Recall per class: [0.91 0.97 0.95 0.97]
F1 Score per class: [0.94 0.96 0.86 0.96]


**Analyse Label Divergence:**

In [21]:
# Sample data (replace these with your actual datasets)
preds = dataset["preds"]
labels = dataset["label"]

# Data preparation
transition_counts = {}
for label, pred in zip(labels, preds):
    if label not in transition_counts:
        transition_counts[label] = {}
    if pred not in transition_counts[label]:
        transition_counts[label][pred] = 0
    transition_counts[label][pred] += 1


In [22]:
import pandas as pd

data = []
for label, preds in transition_counts.items():
    for pred, count in preds.items():
        if label != pred:
            data.append({'Label': label, 'Prediction': pred, 'Count': count})
            
# Sort by label and prediction for better readability
df = pd.DataFrame(data)
df.sort_values(by=['Label', 'Prediction'], inplace=True)
print(df)

    Label  Prediction  Count
11      0           1     30
9       0           2    170
10      0           3     22
6       1           0    759
7       1           2     40
8       1           3      5
0       2           0    547
2       2           1      4
1       2           3      1
3       3           0    600
5       3           1      2
4       3           2     53


**Examples with deviant Predictions:**

In [23]:
from tabulate import tabulate

urls = list(grouped_dataset.keys())
chunks = list(grouped_dataset.values())
table_data = []  # List to store all row data for tabulation

for id_x, (label, prediction) in enumerate(zip(labels_for_url, prediction_for_url_max)):
    if label !=  0 and prediction == 0:
        # Display only up to n chunks, each with detailed information
        for chunk_id, chunk in enumerate(chunks[id_x][:25]):
            preds = chunk['preds']
            text = chunk['text']
            token_count = len(text.split(" "))

            # Limiting text output to the first 50 and last 50 tokens
            text_preview = (text[:50] + '...' + text[-50:]) if len(text) > 100 else text

            # Append each chunk as a separate row in the table
            table_data.append([
                id_x + 1,  # Example ID
                urls[id_x][:250],
                id_to_class[label],
                id_to_class[prediction],
                chunk_id + 1,
                preds,
                token_count,
                text
            ])

        # Break the loop 
        if len(table_data) >= 10: 
            break

# Formatting the output using tabulate
headers = ['Example ID', 'URL', 'Label', 'Prediction', 'Chunk ID', 'Predictions', 'Token Count', 'Text Preview']
display(tabulate(table_data, headers=headers, tablefmt='html', maxcolwidths=[5, 10, 5,  5, 5, 5, 5, 45]))


Example ID,URL,Label,Prediction,Chunk ID,Predictions,Token Count,Text Preview
50,bundestag. de/resourc e/blob/929 418/990d53 6e8b9dd7be 3a12ff0bf0 e1a0f2/WD- 9-083-22-p df- data.pdf,kinde r,other,1,0,1,DECODING/TERROR
50,bundestag. de/resourc e/blob/929 418/990d53 6e8b9dd7be 3a12ff0bf0 e1a0f2/WD- 9-083-22-p df- data.pdf,kinde r,other,2,0,1,DECODING_ERROR
99,bundesgesu ndheitsmin isterium.d e/fileadmi n/Dateien/ 3_Download s/Gesetze_ und_Verord nungen/GuV /C/Kabinet tvorlage_E ckpunktepa pier_Abgab e_Cannabis .pdf,canna bis,other,1,0,1,DECODING_ERROR
112,dip.bundes tag.de/vor gang/.../1 1766,energ ie,other,1,0,23,"If you're seeing this message, that means JavaScript has been disabled on your browser , please enable JS to make this app work."
191,zeit.de/po litik/deut schland/20 23-02/koal ition-kind ergrundsic herung- streit- cdu- christian- lindner- ricarda-la ng?utm_ref errer=http s%3A%2F%2F www.google .com%2F,kinde r,other,1,0,173,"Zum Inhalt springen BOA Berufstest Studium- Interessentest Studienorientierung: Die Suchmaschine für Studiengänge ZEIT Campus ZEIT ONLINE Wie wollen Sie zeit.de nutzen? zeit.de mit Werbung Um der Nutzung mit Werbung zuzustimmen, muss JavaScript in Ihrem Browser aktiviert sein. zeit.de mit Werbung Besuchen Sie zeit.de wie gewohnt mit Werbung und Tracking. Mit Ihrer Zustimmung speichern und verarbeiten wir und unsere Partner Cookies und andere Technologien auf Ihrem Gerät sowie personenbezogene Daten, um unser Webangebot zu verbessern und zu finanzieren. In der Datenschutzerklärung und im Privacy Center finden Sie weitere Details. Ihre Zustimmung ist jederzeit über den Link Privacy Einstellungen am Ende jeder Seite widerrufbar. zeit.de Pur - werbefrei lesen Nutzen Sie zeit.de nahezu ohne Werbung und ohne Werbetracking. zeit.de Pur - werbefrei lesen: Jetzt abonnieren Bereits Pur abonniert? Hier anmelden . Für die Nutzung mit Werbung: Wir erheben personenbezogene Daten und übermitteln diese auch an Drittanbieter , die uns helfen, unser Webangebot zu verbessern und zu finanzieren. Eine Verarbeitung der auf Ihrem Gerät gespeicherten Informationen wie z.B. Cookies (Rechtsgrundlage: § 25 Abs. 1 TTDSG i.V.m. Art"
191,zeit.de/po litik/deut schland/20 23-02/koal ition-kind ergrundsic herung- streit- cdu- christian- lindner- ricarda-la ng?utm_ref errer=http s%3A%2F%2F www.google .com%2F,kinde r,other,2,0,144,".B. Cookies (Rechtsgrundlage: § 25 Abs. 1 TTDSG i.V.m. Art. 6 Abs. 1 lit. a DSGVO) oder persönliche Identifikatoren, IP-Adressen sowie Ihres individuellen Nutzungsverhaltens (Rechtsgrundlagen: Art. 6 Abs. 1 lit. a und f DSGVO) erfolgt dabei zu den folgenden Zwecken: Informationen auf einem Gerät speichern und/oder abrufen Für die Ihnen angezeigten Verarbeitungszwecke können Cookies, Geräte-Kennungen oder andere Informationen auf Ihrem Gerät gespeichert oder abgerufen werden. Personalisierte Anzeigen und Inhalte, Anzeigen- und Inhaltsmessungen, Erkenntnisse über Zielgruppen und Produktentwicklungen Anzeigen und Inhalte können basierend auf einem Profil personalisiert werden. Es können mehr Daten hinzugefügt werden, um Anzeigen und Inhalte besser zu personalisieren. Die Performance von Anzeigen und Inhalten kann gemessen werden. Erkenntnisse über Zielgruppen, die die Anzeigen und Inhalte betrachtet haben, können abgeleitet werden. Daten können verwendet werden, um Benutzerfreundlichkeit, Systeme und Software aufzubauen oder zu verbessern. Pur-Abo FAQ Impressum AGB Datenschutz Switch to english version"
304,zeit.de/zu stimmung?u rl=https%3 A%2F%2Fwww .zeit.de%2 Fpolitik%2 Fdeutschla nd%2F2023- 02%2Fkoali tion-kinde rgrundsich erung- streit- cdu- christian- lindner- ricarda- lang,kinde r,other,1,0,181,"Zum Inhalt springen BOA Berufstest Online StudienInteressen zur Studienfinanzierung: Eine Testfunktion der Studiengänge ZEIT ONLINE ZEIT ONLINE Wie wollen Sie Zeit.de verwenden? Zeit.de ohne Cookies Um der Verwendung mit Werbung zuzustimmen Sie sollte JavaScript in Ihrem Browser neu installiert werden! zeit24.

## Filter Training Set

In [27]:
filtered_dataset = dataset.filter(lambda example: example['label'] == example['preds'] or example['label'] == 0)

Filter: 100%|██████████| 15828/15828 [00:06<00:00, 2303.18 examples/s]


In [28]:
dataset

Dataset({
    features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'token_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'chunk_id', 'url_path', 'input_ids', 'attention_mask', 'probas', 'preds'],
    num_rows: 15828
})

In [29]:
filtered_dataset

Dataset({
    features: ['_id', 'batch_id', 'domain', 'view_url', 'lang', 'text', 'text_length', 'word_count', 'token_count', 'topic', 'category', 'good_for_training', 'good_for_augmentation', 'annotation_type', 'is_topic', 'label', 'chunk_id', 'url_path', 'input_ids', 'attention_mask', 'probas', 'preds'],
    num_rows: 13817
})

In [30]:
MAX_CONTENT_LENGTH = 384
file_path = f"../data/tmp/processed_dataset_multiclass_chunkified_{MAX_CONTENT_LENGTH}"
dataset = load_from_disk(file_path)

#dataset = dataset["train"]

In [31]:
dataset["train"] = filtered_dataset

In [32]:
dataset.save_to_disk(file_path + "_filtered")

Saving the dataset (1/1 shards): 100%|██████████| 827/827 [00:00<00:00, 112733.25 examples/s]
